In [ ]:
import mlflow
import os
import datetime
import mlflow
from mlflow.tracking import MlflowClient
import pandas as pd
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score

In [ ]:
client = MlflowClient()

### Работа с экспериментами

In [ ]:
client.list_experiments()

In [ ]:
exp = client.get_experiment_by_name('driver-accident')
exp

In [ ]:
client.list_run_infos(exp.experiment_id)

In [ ]:
run_info = client.list_run_infos(exp.experiment_id)[0]
run_info

In [ ]:
run_id = run_info.run_id

In [ ]:
run = client.get_run(run_id)
run

In [ ]:
run.data.metrics

In [ ]:
run.data.params

### Работа с моделями

In [ ]:
last_models = client.list_registered_models()
reg_model = last_models[0]
reg_model

In [ ]:
reg_model.latest_versions

In [ ]:
def get_last_prod_model(name):
    last_models = client.get_registered_model(name).latest_versions
    models = list(filter(lambda x: x.current_stage == 'Production', last_models))
    if len(models) == 0:
        return None
    else:
        return models[0]

In [ ]:
model_version = get_last_prod_model('driver-accident')
model_version

In [ ]:
m_version = model_version.version
m_version

### Подготовка данных

In [ ]:
df = pd.read_parquet('driver-stat.parquet')

In [ ]:
FEATURES = ['age', 'sex', 'car_class', 'driving_experience', 'speeding_penalties', 'parking_penalties', 'total_car_accident']

In [ ]:
test_df = df[FEATURES]

### Загрузка и применение модели

In [ ]:
#logged_model = 'runs:/26b40e069ae34d82b35999f81671b606/driver-accident'
logged_model = f'models:/driver-accident/{m_version}'

loaded_model = mlflow.pyfunc.load_model(logged_model)

import pandas as pd
predict = loaded_model.predict(test_df)

In [ ]:
roc_auc = roc_auc_score(df.has_car_accident, predict)
precision = precision_score(df.has_car_accident, predict)
recall = recall_score(df.has_car_accident, predict)
f1 = f1_score(df.has_car_accident, predict)

print(f'ROC AUC: {roc_auc}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1: {f1}')

### Сервировка модели с MLFlow

In [ ]:
!mlflow models serve -m models:/driver-accident/8 --port 10201

In [ ]:
!curl -d '{"columns":["age", "sex", "car_class", "driving_experience", "speeding_penalties", "parking_penalties", "total_car_accident"], "data":[[18,"male","A",5,5,1,0]]}' -H 'Content-Type: application/json; format=pandas-split' -X POST localhost:10201/invocations